In [26]:
from http.server import SimpleHTTPRequestHandler, HTTPServer
from urllib import request, error
import pandas as pd
from pycaret.clustering import *
import urllib.parse
import sys


In [27]:
http = pd.read_csv("sample.csv")
http

,method,path,body,single_q,double_q,dashes,braces,spaces,badwords,class
0,POST,/products/view?id=1,uname=test&pass=test,0,0,0,0,0,0,1
1,GET,/languages/english?q=%3Cimg+src%3Dx+onerror%3D...,NaN,0,0,0,1,2,1,0
2,POST,/locations/usa,comment=Nice+product,0,0,0,0,1,0,1
3,POST,/tickets?q=1%3B+DROP+TABLE+users%3B,data=1%3B+DROP+TABLE+users%3B,0,0,0,0,6,4,0
4,POST,/crypto,searchFor=book&goButton=go,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...
9995,GET,/search?q=laptops,NaN,0,0,0,0,0,0,1
9996,POST,/travel/hotels,NaN,0,0,0,0,0,0,1
9997,GET,/api/v3/dashboard,NaN,0,0,0,0,0,0,1
9998,GET,/auth/verify-email,NaN,0,0,0,0,0,0,1


In [28]:
clu1=setup(data= http, normalize=True, numeric_features=['single_q','double_q','dashes','braces','spaces','badwords'], ignore_features = ['method','path','body','class'])

,Description,Value
0,Session id,5532
1,Original data shape,"(10000, 10)"
2,Transformed data shape,"(10000, 6)"
3,Ignore features,4
4,Numeric features,6
5,Rows with missing values,55.2%
6,Preprocess,True
7,Imputation type,simple
8,Numeric imputation,mean
9,Categorical imputation,mode


In [29]:
kmeans = create_model('kmeans',num_clusters=2)

,Silhouette,Calinski-Harabasz,Davies-Bouldin,Homogeneity,Rand Index,Completeness
0,0.6904,3906.2581,0.9315,0,0,0


In [30]:
kmeans

KMeans(n_clusters=2, random_state=5532)

In [31]:
plot_model(kmeans)

In [32]:
kmeans_result = assign_model(kmeans)
# kmeans_result.tail(50)
kmeans_result.to_csv("result.csv")

In [33]:
test_http = pd.read_csv("test.csv")
test_http.head()

,method,path,body,single_q,double_q,dashes,braces,spaces,badwords
0,POST,/products/view?id=1,uname=test&pass=test,0,0,0,0,0,0
1,GET,/languages/english?q=%3Cimg+src%3Dx+onerror%3D...,NaN,0,0,0,1,2,1
2,POST,/locations/usa,comment=Nice+product,0,0,0,0,1,0
3,POST,/tickets?q=1%3B+DROP+TABLE+users%3B,data=1%3B+DROP+TABLE+users%3B,0,0,0,0,6,4
4,POST,/crypto,searchFor=book&goButton=go,0,0,0,0,0,0


In [34]:
test_result = predict_model(kmeans,data=test_http)

In [35]:
test_result

,single_q,double_q,dashes,braces,spaces,badwords,Cluster
0,-0.276391,-0.169266,-0.262788,-0.312674,-0.462716,-0.509434,Cluster 1
1,-0.276391,-0.169266,-0.262788,1.459855,0.756080,0.145029,Cluster 1
2,-0.276391,-0.169266,-0.262788,-0.312674,0.146682,-0.509434,Cluster 1
3,-0.276391,-0.169266,-0.262788,-0.312674,3.193671,2.108418,Cluster 1
4,-0.276391,-0.169266,-0.262788,-0.312674,-0.462716,-0.509434,Cluster 1
...,...,...,...,...,...,...,...
9995,-0.276391,-0.169266,-0.262788,-0.312674,-0.462716,-0.509434,Cluster 1
9996,-0.276391,-0.169266,-0.262788,-0.312674,-0.462716,-0.509434,Cluster 1
9997,-0.276391,-0.169266,-0.262788,-0.312674,-0.462716,-0.509434,Cluster 1
9998,-0.276391,-0.169266,-0.262788,-0.312674,-0.462716,-0.509434,Cluster 1


In [36]:
test_result.to_csv("test_result.csv")

In [83]:
badwords = [
    'sleep','drop','uid','select','waitfor','delay','system',
    'union','order by','group by','insert','update','delete',
    'benchmark','and 1=1','or 1=1'
]

def ExtractFeatures(method,path,body):
    # print(method,path,body)
    path = urllib.parse.unquote(path)
    badwords_count = 0
    single_q = path.count("'")
    double_q = path.count("\"")
    dashes = path.count("--")
    braces = path.count("(")
    spaces = path.count(" ")
    for word in badwords:
        badwords_count += path.count(word)
        
    lst = [single_q,double_q,dashes,braces,spaces,badwords_count]
    return pd.DataFrame([lst],columns = ["single_q","double_q","dashes","braces","spaces","badwords"])

In [38]:
# from http.server import SimpleHTTPRequestHandler, HTTPServer
# from urllib import request, error
# import pandas as pd
# from pycaret.clustering import *
# import urllib.parse
# import sys

# def classify_request(single_q, double_q, dashes, braces, spaces, badwords_count, threshold=6):
#     score = 0
#     score += badwords_count * 3
#     score += single_q * 1
#     score += double_q * 1
#     score += dashes * 2
#     score += braces * 1

#     # Your logic: suspicious -> 0, normal -> 1
#     return 0 if score >= threshold else 1

# def ExtractFeatures(method, path_enc, body_enc, headers):
#     # Ensure inputs are strings (not None)
#     method = method or ""
#     path_enc = path_enc or ""
#     body_enc = body_enc or ""
#     headers = headers or {}

#     # decode URL-encoding for counting (do not re-encode)
#     path = unquote_plus(path_enc)
#     body = unquote(body_enc)

#     single_q = path.count("'") + body.count("'")
#     double_q = path.count('"') + body.count('"')
#     dashes = path.count("--") + body.count("--")
#     braces = path.count("(") + body.count("(")
#     spaces = path.count(" ") + body.count(" ")

#     # badwords counting (case-insensitive)
#     badwords_count = 0
#     for word in badwords:
#         if not word:
#             continue
#         badwords_count += path.lower().count(word.lower())
#         badwords_count += body.lower().count(word.lower())

#     # count badwords in headers too
#     for hname, hval in headers.items():
#         val = (hval or "").lower()
#         for word in badwords:
#             badwords_count += val.count(word.lower())

#     # classify: returns int 0 or 1
#     class_value = classify_request(single_q, double_q, dashes, braces, spaces, badwords_count)

#     # Return types: keep path & body AS STRINGS (no bytes)
#     return [
#         method,
#         path_enc.strip(),
#         body_enc.strip(),
#         single_q,
#         double_q,
#         dashes,
#         braces,
#         spaces,
#         badwords_count,
#         class_value
#     ]

In [88]:
# from http.server import SimpleHTTPRequestHandler, HTTPServer
class SimpleHTTPProxy(SimpleHTTPRequestHandler):
    proxy_routes ={}

    @classmethod
    def set_routes(cls, proxy_routes):
        cls.proxy_routes = proxy_routes

    def do_GET(self):
        parts = self.path.split("/")
        # print(parts)
        live_data = ExtractFeatures("GET",parts[3],"")
        print(live_data)
        result = predict_model(kmeans,data = live_data)
        print("result data=",result)
        print(result['Cluster'][0])
        if result['Cluster'][0] == 'Cluster 1':
            print("Intrusion Detcted")

        if len(parts) >=2:
            self.proxy_request('http://'+parts[2]+'/')
        else:
            super().do_GET()

    def proxy_request(self, url):
        try:
            response = request.urlopen(url)
        except error.HTTPError as e:
            # print('err')
            self.send_response_only(e.code)
            self.end_headers()
            return
        self.send_response_only(response.status)
        for name,value in response.headers.items():
            self.send_header(name,value)
        self.end_headers()
        self.copyfile(response, self.wfile)
        
SimpleHTTPProxy.set_routes({'proxy_route':'http://demo.testfire.net/'})
with HTTPServer(('127.0.0.1',8080),SimpleHTTPProxy) as httpd:
    host, port = httpd.socket.getsockname()
    print(f'listining http://{host}:{port}')
    try:
        httpd.serve_forever()
    except KeyboardInterrupt:
        print("Keyboard interrupt")

listining http://127.0.0.1:8080
   single_q  double_q  dashes  braces  spaces  badwords
0         0         0       0       0       0         0
result data=    single_q  double_q    dashes    braces    spaces  badwords    Cluster
0 -0.276391 -0.169266 -0.262788 -0.312674 -0.462716 -0.509434  Cluster 1
Cluster 1
Intrusion Detcted
   single_q  double_q  dashes  braces  spaces  badwords
0         0         0       0       0       0         0
result data=    single_q  double_q    dashes    braces    spaces  badwords    Cluster
0 -0.276391 -0.169266 -0.262788 -0.312674 -0.462716 -0.509434  Cluster 1
Cluster 1
Intrusion Detcted


127.0.0.1 - - [18/Nov/2025 13:49:03] code 501, message Unsupported method ('CONNECT')
127.0.0.1 - - [18/Nov/2025 13:49:03] "CONNECT push.services.mozilla.com:443 HTTP/1.1" 501 -
127.0.0.1 - - [18/Nov/2025 13:49:03] code 501, message Unsupported method ('CONNECT')
127.0.0.1 - - [18/Nov/2025 13:49:03] "CONNECT sync-1-us-west1-g.sync.services.mozilla.com:443 HTTP/1.1" 501 -
127.0.0.1 - - [18/Nov/2025 13:49:03] code 501, message Unsupported method ('CONNECT')
127.0.0.1 - - [18/Nov/2025 13:49:03] "CONNECT api.accounts.firefox.com:443 HTTP/1.1" 501 -
127.0.0.1 - - [18/Nov/2025 13:49:03] code 501, message Unsupported method ('CONNECT')
127.0.0.1 - - [18/Nov/2025 13:49:03] "CONNECT incoming.telemetry.mozilla.org:443 HTTP/1.1" 501 -
127.0.0.1 - - [18/Nov/2025 13:49:03] code 501, message Unsupported method ('CONNECT')
127.0.0.1 - - [18/Nov/2025 13:49:03] "CONNECT mirror.toolbar.netcraft.com:443 HTTP/1.1" 501 -
127.0.0.1 - - [18/Nov/2025 13:49:03] code 501, message Unsupported method ('CONNECT')


   single_q  double_q  dashes  braces  spaces  badwords
0         0         0       0       0       0         0
result data=    single_q  double_q    dashes    braces    spaces  badwords    Cluster
0 -0.276391 -0.169266 -0.262788 -0.312674 -0.462716 -0.509434  Cluster 1
Cluster 1
Intrusion Detcted


127.0.0.1 - - [18/Nov/2025 13:49:04] code 501, message Unsupported method ('CONNECT')
127.0.0.1 - - [18/Nov/2025 13:49:04] "CONNECT api.web3auth.io:443 HTTP/1.1" 501 -
127.0.0.1 - - [18/Nov/2025 13:49:04] code 501, message Unsupported method ('CONNECT')
127.0.0.1 - - [18/Nov/2025 13:49:04] "CONNECT incoming.telemetry.mozilla.org:443 HTTP/1.1" 501 -
127.0.0.1 - - [18/Nov/2025 13:49:04] code 501, message Unsupported method ('CONNECT')
127.0.0.1 - - [18/Nov/2025 13:49:04] "CONNECT metamask.github.io:443 HTTP/1.1" 501 -
127.0.0.1 - - [18/Nov/2025 13:49:04] code 501, message Unsupported method ('CONNECT')
127.0.0.1 - - [18/Nov/2025 13:49:04] "CONNECT execution.metamask.io:443 HTTP/1.1" 501 -
127.0.0.1 - - [18/Nov/2025 13:49:04] code 501, message Unsupported method ('CONNECT')
127.0.0.1 - - [18/Nov/2025 13:49:04] "CONNECT execution.metamask.io:443 HTTP/1.1" 501 -
127.0.0.1 - - [18/Nov/2025 13:49:10] code 501, message Unsupported method ('CONNECT')
127.0.0.1 - - [18/Nov/2025 13:49:10] "CONNEC

   single_q  double_q  dashes  braces  spaces  badwords
0         0         0       0       0       0         0
result data=    single_q  double_q    dashes    braces    spaces  badwords    Cluster
0 -0.276391 -0.169266 -0.262788 -0.312674 -0.462716 -0.509434  Cluster 1
Cluster 1
Intrusion Detcted
   single_q  double_q  dashes  braces  spaces  badwords
0         0         0       0       0       0         0
result data=    single_q  double_q    dashes    braces    spaces  badwords    Cluster
0 -0.276391 -0.169266 -0.262788 -0.312674 -0.462716 -0.509434  Cluster 1
Cluster 1
Intrusion Detcted
   single_q  double_q  dashes  braces  spaces  badwords
0         0         0       0       0       0         0
result data=    single_q  double_q    dashes    braces    spaces  badwords    Cluster
0 -0.276391 -0.169266 -0.262788 -0.312674 -0.462716 -0.509434  Cluster 1
Cluster 1
Intrusion Detcted


...

   single_q  double_q  dashes  braces  spaces  badwords
0         0         0       0       0       0         0
result data=    single_q  double_q    dashes    braces    spaces  badwords    Cluster
0 -0.276391 -0.169266 -0.262788 -0.312674 -0.462716 -0.509434  Cluster 1
Cluster 1
Intrusion Detcted
   single_q  double_q  dashes  braces  spaces  badwords
0         0         0       0       0       0         0
result data=    single_q  double_q    dashes    braces    spaces  badwords    Cluster
0 -0.276391 -0.169266 -0.262788 -0.312674 -0.462716 -0.509434  Cluster 1
Cluster 1
Intrusion Detcted


127.0.0.1 - - [18/Nov/2025 14:25:00] code 501, message Unsupported method ('CONNECT')
127.0.0.1 - - [18/Nov/2025 14:25:00] "CONNECT execution.metamask.io:443 HTTP/1.1" 501 -
127.0.0.1 - - [18/Nov/2025 14:26:05] code 501, message Unsupported method ('CONNECT')
127.0.0.1 - - [18/Nov/2025 14:26:05] "CONNECT execution.metamask.io:443 HTTP/1.1" 501 -
127.0.0.1 - - [18/Nov/2025 14:26:25] code 501, message Unsupported method ('CONNECT')
127.0.0.1 - - [18/Nov/2025 14:26:25] "CONNECT ads.mozilla.org:443 HTTP/1.1" 501 -
127.0.0.1 - - [18/Nov/2025 14:26:26] code 501, message Unsupported method ('CONNECT')
127.0.0.1 - - [18/Nov/2025 14:26:26] "CONNECT firefox.settings.services.mozilla.com:443 HTTP/1.1" 501 -


   single_q  double_q  dashes  braces  spaces  badwords
0         0         0       0       0       0         0
result data=    single_q  double_q    dashes    braces    spaces  badwords    Cluster
0 -0.276391 -0.169266 -0.262788 -0.312674 -0.462716 -0.509434  Cluster 1
Cluster 1
Intrusion Detcted
   single_q  double_q  dashes  braces  spaces  badwords
0         0         0       0       0       0         0
result data=    single_q  double_q    dashes    braces    spaces  badwords    Cluster
0 -0.276391 -0.169266 -0.262788 -0.312674 -0.462716 -0.509434  Cluster 1
Cluster 1
Intrusion Detcted


127.0.0.1 - - [18/Nov/2025 14:27:20] code 501, message Unsupported method ('CONNECT')
127.0.0.1 - - [18/Nov/2025 14:27:20] "CONNECT execution.metamask.io:443 HTTP/1.1" 501 -


Keyboard interrupt
